In [13]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
import torchmetrics

In [14]:
class MLP(torch.nn.Module):
    def __init__(self, n_hidden_nodes, n_classes, image_width=32, image_height=32, color_channels=3):
        super().__init__()
        self.n_hidden_nodes = n_hidden_nodes
        self.n_classes = n_classes
        
        self.relu = nn.ReLU6()
        self.leaky_relu = nn.LeakyReLU()
        
        self.image_width = image_width
        self.image_height = image_height
        self.color_channels = color_channels
        self.input_size = self.image_width * self.image_height * self.color_channels
        
        # define layers
        self.hidden_layer = nn.Linear(self.input_size, self.n_hidden_nodes) # (B, c * w * h) -> (B, 100)
        self.hidden_layer_2 = nn.Linear(self.n_hidden_nodes, self.n_hidden_nodes) # (B, 100) -> (B, 100)
        self.output_layer = nn.Linear(self.n_hidden_nodes, self.n_classes) # (B, 100) -> (B, nclasses)
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.relu(self.hidden_layer(x))
        x = self.leaky_relu(self.hidden_layer_2(x))
        x = self.leaky_relu(self.hidden_layer_2(x))
        x = self.output_layer(x)
        return x


In [15]:
class Data():
    def __init__(self, root='./data', transform=None, batch_size=64, num_workers=2):
        self.root = root
        if transform is not None:
            self.transform = transform
        else:
            self.transform = transforms.Compose([
                        transforms.RandomHorizontalFlip(),
                        transforms.RandomCrop(32, padding=4),
                        transforms.ToTensor(),
                        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])
        self.batch_size = batch_size
        self.num_workers = num_workers
    
    def load_train_data(self):
        train_dataset = torchvision.datasets.CIFAR10(root=self.root, train=True, download=True, transform=self.transform)

        return DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers)

    def load_test_data(self):
        test_dataset = torchvision.datasets.CIFAR10(root=self.root, train=False, download=True, transform=self.transform)

        return DataLoader(test_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)    

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [21]:
class Model():
    def __init__(self, model=None, criterion=None, optimizer=None, dataloader=None, epochs=10):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        if model is not None:
            self.model = model
        else:
            self.model = MLP(n_classes=10, n_hidden_nodes=100, image_width=32, image_height=32, color_channels=3)
        self.model.to(self.device)
        
        if criterion is not None:
            self.criterion = criterion
        else:
            self.criterion = CrossEntropyLoss()
        
        if optimizer is not None:
            self.optimizer = optimizer
        else:
            self.optimizer = SGD(self.model.parameters(), lr=0.005)
        
        if dataloader is not None:
            self.dataloader = dataloader
        else:
            self.dataloader = Data(root='./data', batch_size=32, num_workers=4)
            
        self.epochs = epochs
    
    def train(self):
        train_loader = self.dataloader.load_train_data()
        
        train_accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=10).to(self.device)

        for epoch in range(self.epochs):
            running_loss = 0.0

            self.model.train()
            train_accuracy.reset()

            for inputs, labels in train_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                
                self.optimizer.zero_grad()
                
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                
                loss.backward()
                self.optimizer.step()

                running_loss += loss.item()

                _, predicted = torch.max(outputs.data, 1)
                train_accuracy.update(predicted, labels)

            final_train_accuracy = train_accuracy.compute()

            print(f'Epoch [{epoch+1}/{self.epochs}]\n', 
                    f'Loss: {running_loss/len(train_loader):.4f}\n',
                    f'Train Accuracy: {final_train_accuracy * 100:.2f}\n',
                    '--------------------------------------------------\n')

        print('======================Finished=========================')
        
    def evaluate(self):
        test_loader = self.dataloader.load_test_data()
        test_accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=10).to(self.device)
        
        self.model.eval()
        test_accuracy.reset()
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                outputs = self.model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                test_accuracy.update(predicted, labels)

        final_test_accuracy = test_accuracy.compute()

        print(f'Test Accuracy: {final_test_accuracy * 100:.2f}\n',
                '--------------------------------------------------\n')

        print('======================Finished=========================')
        
        
    def predict(self, data):
        prediction = []
        
        self.model.eval()
        with torch.no_grad():
            for input in data:
                input = input.to(self.device)
                outputs = self.model(input)
                _, predicted = torch.max(outputs.data, 1)
                prediction.append(predicted)

        return prediction    
    
    def save(self, name=None):
        parent = 'models'
        checkpoint_path = name if name is not None else 'mlp_checkpoint.pth'
        path = parent + '/' + checkpoint_path

        checkpoint = {
            'epoch': self.epochs,
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'criterion_state_dict': self.criterion.state_dict(),
        }

        torch.save(checkpoint, path)
        print(f"Checkpoint saved to {path}")


In [22]:
clf = Model()
clf.train()

Files already downloaded and verified
Epoch [1/10]
 Loss: 2.2228
 Train Accuracy: 17.51
 --------------------------------------------------

Epoch [2/10]
 Loss: 1.9993
 Train Accuracy: 27.22
 --------------------------------------------------

Epoch [3/10]
 Loss: 1.9010
 Train Accuracy: 31.35
 --------------------------------------------------

Epoch [4/10]
 Loss: 1.8238
 Train Accuracy: 34.10
 --------------------------------------------------

Epoch [5/10]
 Loss: 1.7690
 Train Accuracy: 36.27
 --------------------------------------------------

Epoch [6/10]
 Loss: 1.7232
 Train Accuracy: 37.81
 --------------------------------------------------

Epoch [7/10]
 Loss: 1.6817
 Train Accuracy: 39.35
 --------------------------------------------------

Epoch [8/10]
 Loss: 1.6489
 Train Accuracy: 40.38
 --------------------------------------------------

Epoch [9/10]
 Loss: 1.6233
 Train Accuracy: 41.59
 --------------------------------------------------

Epoch [10/10]
 Loss: 1.5983
 Train 

In [23]:
clf.evaluate()

Files already downloaded and verified
Test Accuracy: 43.32
 --------------------------------------------------

======================Finished=========================


In [24]:
clf.save('mlp_checkpoint.pth')


Checkpoint saved to models/mlp_checkpoint.pth
